In [2]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, CosineAnnealingWarmRestarts
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds
from tools import EarlyStopping

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2
1.3.1
GeForce RTX 2070 SUPER


device(type='cuda', index=0)

In [3]:
train_folder = 'data/train/'
test_folder = 'data/test/'
train_label_path = 'data/train_label.csv'

In [4]:
train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [5]:
num_class = len(train_label['label'].unique())

In [6]:
# 모든 csv 파일의 상태_B로 변화는 시점이 같다라고 가정
# 하지만, 개별 csv파일의 상태_B로 변화는 시점은 상이할 수 있음
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close()
        pool.join()        
    combined_df = pd.concat(df_list)    
    return combined_df

In [7]:
# train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, 
#                            event_time=10, nrows=120)


In [8]:
train = joblib.load('data/df_train_10_60.pkl').reset_index()
test = joblib.load('data/df_test_10.pkl')


In [9]:
y_cols = 'label'
fea_cols = [c for c in train.columns if c[0] == 'V']
len(fea_cols), y_cols

(5121, 'label')

In [10]:
scaler = joblib.load('scaler_20200129T135731.bin')

train[fea_cols] = scaler.transform(train[fea_cols].values)
test[fea_cols] = scaler.transform(test[fea_cols].values)

In [11]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=81511991154 % 2**32-1)

# X_train.shape, X_test.shape

In [12]:
class Dataset15(Dataset):
    def __init__(self, df, fea_cols, y_cols):        
        self.X = df[fea_cols].values
#         self.y = pd.get_dummies(df[y_cols]).values
        self.y = df[y_cols].values

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.long)
    

In [13]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler, device):
        self.device = device
        self.model = model#.to(self.device)
        self.criterion = criterion#.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler

        print(self.model.train())
        pass
    
    def train(self, data_loader, epoch):
        self.model.train()
        total_loss = 0
        
        for i, data in enumerate(data_loader):
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            self.scheduler.step(epoch + i / len(data_loader))
            self.optimizer.zero_grad()
            y_pred = self.model(X_batch)
#             print(y_pred, y_batch)
            
            loss = self.criterion(y_pred, y_batch)
            total_loss = total_loss + loss.item()
            
            loss.backward()
            self.optimizer.step()
        
        
        return total_loss / len(data_loader)
    
    def eval(self, data_loader):
        self.model.eval()
        total_loss = 0
#         print('valid_loader', len(valid_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            with torch.no_grad():
                y_pred = self.model(X_batch)
                loss = self.criterion(y_pred, y_batch)
                total_loss = total_loss + loss.item()
        return total_loss / len(data_loader)

    def save(self, model_path='checkpoint.pt'):
#         torch.save(self.model.state_dict(), 'checkpoint.pt')
        joblib.dump(self.model, model_path)
        return
    
    def load(self, model_path='checkpoint.pt'):
#         self.model.load_state_dict(torch.load(model_path))
        self.model = joblib.load(model_path)
        return

#### Model

In [14]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024), act, dropout,
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout, 
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout, 
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout, 
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout, 
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout, 
            torch.nn.Linear(1024, 198)
        )
    def forward(self, x):
        return self.model(x)
 

In [15]:
class CNNModel(torch.nn.Module):
    def __init__(self, dropout_probability=0.5):
        super().__init__()
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.cnn = torch.nn.Sequential(
            torch.nn.Conv1d(1, 2, 3, stride=1, padding=1), torch.nn.BatchNorm1d(2), relu,
            torch.nn.Conv1d(2, 4, 3, stride=1, padding=1), torch.nn.BatchNorm1d(4), relu,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(4, 4, 3, stride=1, padding=1), torch.nn.BatchNorm1d(4), relu,
            torch.nn.Conv1d(4, 8, 3, stride=1, padding=1), torch.nn.BatchNorm1d(8), relu,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(8, 8, 3, stride=1, padding=1), torch.nn.BatchNorm1d(8), relu,
            torch.nn.Conv1d(8, 8, 3, stride=1, padding=1), torch.nn.BatchNorm1d(8), relu,
            torch.nn.MaxPool1d(2),
#             torch.nn.Conv1d(48, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), relu,
#             torch.nn.Conv1d(96, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), relu,
#             torch.nn.MaxPool1d(2),
#             torch.nn.Conv1d(96, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), relu,
#             torch.nn.Conv1d(96, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), relu,
#             torch.nn.MaxPool1d(2),
        )
            
        self.clf = torch.nn.Sequential(
            torch.nn.Linear(5120, 3000), torch.nn.BatchNorm1d(3000), relu, dropout,
            torch.nn.Linear(3000, 1024), torch.nn.BatchNorm1d(1024), relu, dropout,
            torch.nn.Linear(1024, 198)
        )
        
    def forward(self, x):
        x = x.unsqueeze(1)
        out = self.cnn(x)
        dim = 1
        for d in out.size()[1:]: #24, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.clf(out)
        return out



In [23]:
class AutoEncoder(nn.Module):
    def __init__(self, input_size, output_size=128):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 2048), torch.nn.BatchNorm1d(2048),#nn.ReLU(True),
            nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048),#nn.ReLU(True), 
            nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048),#nn.ReLU(True), 
            nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024),#nn.ReLU(True), 
            nn.Linear(1024, 512), torch.nn.BatchNorm1d(512),#nn.ReLU(True), 
            nn.Linear(512, 256), torch.nn.BatchNorm1d(256),#nn.ReLU(True), 
            nn.Linear(256, output_size), nn.Sigmoid()
        )
        self.decoder = nn.Sequential(
            nn.Linear(output_size, 256), torch.nn.BatchNorm1d(256),#nn.ReLU(True),
            nn.Linear(256, 512), torch.nn.BatchNorm1d(512),#nn.ReLU(True),
            nn.Linear(512, 1024), torch.nn.BatchNorm1d(1024),#nn.ReLU(True), 
            nn.Linear(1024, 2048), torch.nn.BatchNorm1d(2048),#nn.ReLU(True), 
            nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048),#nn.ReLU(True), 
            nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048),#nn.ReLU(True), 
            nn.Linear(2048, input_size), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

#### Train

In [24]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print(model_ts)

# print(f'fea_size {len(fea_cols)} layer_cols {layer_cols}')

torch.manual_seed(81511991154)
torch.initial_seed()

20200207T164850


81511991154

In [25]:
dataset = Dataset15(train[fea_cols + [y_cols]], fea_cols, y_cols)

train_set, val_set = torch.utils.data.random_split(dataset, [int(train.shape[0] * 0.8), int(train.shape[0] * 0.2)])

print(len(train_set), len(val_set))

batch_size = 3000
num_workers = 8

all_loader = DataLoader(dataset=dataset, batch_size=len(dataset) // 2, num_workers=num_workers, shuffle=True)
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=len(val_set))

print(f'batch_size {batch_size} num_workers {num_workers}')
print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

33120 8280
batch_size 3000 num_workers 8
train_loader 12 val_loader 1


#### AE

In [31]:
# model = AutoEncoder(input_size=len(fea_cols), output_size=128).to(device)

total_epoch = 400
patience = 500
val_loss_min = np.Inf

early_stopping = EarlyStopping(patience=patience, min_epoch=1, verbose=True)
early_stopping.val_loss_min = val_loss_min
early_stopping.best_score = None if val_loss_min==np.Inf else -val_loss_min 


# optimizer = torch.optim.SGD(model.parameters(), lr=0.015, momentum=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
# scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=100, eta_min=0.00001)

scheduler = StepLR(optimizer, step_size = 100, gamma=0.5)

criterion = nn.MSELoss(reduction='mean').to(device)


for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
    if os.path.isfile('stop.flag'):
        print(f'{e} stop!')
        break

    model.train()
    total_loss = 0

    for i, data in enumerate(all_loader):
        X_batch, _ = data
        X_batch = X_batch.to(device)

#     scheduler.step(total_epoch + i / len(all_loader))
    
        optimizer.zero_grad()
        y_pred = model(X_batch)

        loss = criterion(y_pred, X_batch)
        total_loss = total_loss + loss.item()

        loss.backward()
        optimizer.step()
        
    scheduler.step()

    train_loss = total_loss / len(all_loader)

    ts = datetime.now().strftime('%Y%m%dT%H%M%S')
    print(f'[{ts}] Epock {e} / {total_epoch}\t lr {scheduler.get_lr()[0]}')
    print(f'  train_loss: {train_loss}')

    early_stopping(train_loss, model)

    if early_stopping.early_stop:
        print("\tEarly stopping epoch {}, valid loss {}".format(e, early_stopping.val_loss_min))
        break



val_loss_min = early_stopping.val_loss_min

model_path = 'model/{}_{}.model'.format(model_ts, val_loss_min)
joblib.dump(model, model_path)
print(model_path)


[20200207T181409] Epock 0 / 400	 lr 0.005
  train_loss: 0.0038892246084287763
Validation loss decreased (inf --> 0.00388922).  Saving model ...
[20200207T181412] Epock 1 / 400	 lr 0.005
  train_loss: 0.004224805044941604
EarlyStopping 2 / 1 counter: 1 out of 500
[20200207T181414] Epock 2 / 400	 lr 0.005
  train_loss: 0.004126828862354159
EarlyStopping 3 / 1 counter: 2 out of 500
[20200207T181417] Epock 3 / 400	 lr 0.005
  train_loss: 0.00396764138713479
EarlyStopping 4 / 1 counter: 3 out of 500
[20200207T181419] Epock 4 / 400	 lr 0.005
  train_loss: 0.003875598544254899
Validation loss decreased (0.00388922 --> 0.00387560).  Saving model ...
[20200207T181422] Epock 5 / 400	 lr 0.005
  train_loss: 0.004188551218248904
EarlyStopping 6 / 1 counter: 1 out of 500
[20200207T181424] Epock 6 / 400	 lr 0.005
  train_loss: 0.003775612683966756
Validation loss decreased (0.00387560 --> 0.00377561).  Saving model ...
[20200207T181427] Epock 7 / 400	 lr 0.005
  train_loss: 0.0039007837185636163
Ear

[20200207T181647] Epock 63 / 400	 lr 0.0025
  train_loss: 0.003507172572426498
EarlyStopping 64 / 1 counter: 1 out of 500
[20200207T181649] Epock 64 / 400	 lr 0.0025
  train_loss: 0.003529568202793598
EarlyStopping 65 / 1 counter: 2 out of 500
[20200207T181652] Epock 65 / 400	 lr 0.0025
  train_loss: 0.0036226403899490833
EarlyStopping 66 / 1 counter: 3 out of 500
[20200207T181654] Epock 66 / 400	 lr 0.0025
  train_loss: 0.0035024541430175304
EarlyStopping 67 / 1 counter: 4 out of 500
[20200207T181656] Epock 67 / 400	 lr 0.0025
  train_loss: 0.003553695511072874
EarlyStopping 68 / 1 counter: 5 out of 500
[20200207T181659] Epock 68 / 400	 lr 0.0025
  train_loss: 0.0034875894198194146
Validation loss decreased (0.00350108 --> 0.00348759).  Saving model ...
[20200207T181702] Epock 69 / 400	 lr 0.0025
  train_loss: 0.0035308286314830184
EarlyStopping 70 / 1 counter: 1 out of 500
[20200207T181704] Epock 70 / 400	 lr 0.0025
  train_loss: 0.003488015732727945
EarlyStopping 71 / 1 counter: 2 o

[20200207T181923] Epock 125 / 400	 lr 0.00125
  train_loss: 0.0033321689115837216
EarlyStopping 126 / 1 counter: 4 out of 500
[20200207T181926] Epock 126 / 400	 lr 0.00125
  train_loss: 0.003327171318233013
EarlyStopping 127 / 1 counter: 5 out of 500
[20200207T181928] Epock 127 / 400	 lr 0.00125
  train_loss: 0.0033008691389113665
Validation loss decreased (0.00332083 --> 0.00330087).  Saving model ...
[20200207T181931] Epock 128 / 400	 lr 0.00125
  train_loss: 0.003311005770228803
EarlyStopping 129 / 1 counter: 1 out of 500
[20200207T181933] Epock 129 / 400	 lr 0.00125
  train_loss: 0.003363437717780471
EarlyStopping 130 / 1 counter: 2 out of 500
[20200207T181936] Epock 130 / 400	 lr 0.00125
  train_loss: 0.0033672467106953263
EarlyStopping 131 / 1 counter: 3 out of 500
[20200207T181938] Epock 131 / 400	 lr 0.00125
  train_loss: 0.0033310374710708857
EarlyStopping 132 / 1 counter: 4 out of 500
[20200207T181940] Epock 132 / 400	 lr 0.00125
  train_loss: 0.0033196735894307494
EarlyStopp

[20200207T182158] Epock 187 / 400	 lr 0.000625
  train_loss: 0.003244122606702149
EarlyStopping 188 / 1 counter: 3 out of 500
[20200207T182200] Epock 188 / 400	 lr 0.000625
  train_loss: 0.003241614787839353
Validation loss decreased (0.00324311 --> 0.00324161).  Saving model ...
[20200207T182203] Epock 189 / 400	 lr 0.000625
  train_loss: 0.0033032416831701994
EarlyStopping 190 / 1 counter: 1 out of 500
[20200207T182205] Epock 190 / 400	 lr 0.000625
  train_loss: 0.0033230334520339966
EarlyStopping 191 / 1 counter: 2 out of 500
[20200207T182208] Epock 191 / 400	 lr 0.000625
  train_loss: 0.003286878578364849
EarlyStopping 192 / 1 counter: 3 out of 500
[20200207T182210] Epock 192 / 400	 lr 0.000625
  train_loss: 0.003255155519582331
EarlyStopping 193 / 1 counter: 4 out of 500
[20200207T182212] Epock 193 / 400	 lr 0.000625
  train_loss: 0.0032691163942217827
EarlyStopping 194 / 1 counter: 5 out of 500
[20200207T182215] Epock 194 / 400	 lr 0.000625
  train_loss: 0.003238717676140368
Vali

[20200207T182431] Epock 248 / 400	 lr 0.0003125
  train_loss: 0.0033088981872424483
EarlyStopping 249 / 1 counter: 3 out of 500
[20200207T182433] Epock 249 / 400	 lr 0.00015625
  train_loss: 0.0032469446305185556
EarlyStopping 250 / 1 counter: 4 out of 500
[20200207T182436] Epock 250 / 400	 lr 0.00015625
  train_loss: 0.0032103584380820394
EarlyStopping 251 / 1 counter: 5 out of 500
[20200207T182438] Epock 251 / 400	 lr 0.00015625
  train_loss: 0.0032208506017923355
EarlyStopping 252 / 1 counter: 6 out of 500
[20200207T182440] Epock 252 / 400	 lr 0.00015625
  train_loss: 0.0032436749897897243
EarlyStopping 253 / 1 counter: 7 out of 500
[20200207T182443] Epock 253 / 400	 lr 0.00015625
  train_loss: 0.0032360790064558387
EarlyStopping 254 / 1 counter: 8 out of 500
[20200207T182445] Epock 254 / 400	 lr 0.00015625
  train_loss: 0.003209805116057396
EarlyStopping 255 / 1 counter: 9 out of 500
[20200207T182447] Epock 255 / 400	 lr 0.00015625
  train_loss: 0.003200023085810244
EarlyStopping 2

[20200207T182703] Epock 311 / 400	 lr 7.8125e-05
  train_loss: 0.003249557572416961
EarlyStopping 312 / 1 counter: 3 out of 500
[20200207T182705] Epock 312 / 400	 lr 7.8125e-05
  train_loss: 0.003193655633367598
EarlyStopping 313 / 1 counter: 4 out of 500
[20200207T182707] Epock 313 / 400	 lr 7.8125e-05
  train_loss: 0.0032380307093262672
EarlyStopping 314 / 1 counter: 5 out of 500
[20200207T182710] Epock 314 / 400	 lr 7.8125e-05
  train_loss: 0.0031989034032449126
EarlyStopping 315 / 1 counter: 6 out of 500
[20200207T182712] Epock 315 / 400	 lr 7.8125e-05
  train_loss: 0.0032519280212000012
EarlyStopping 316 / 1 counter: 7 out of 500
[20200207T182714] Epock 316 / 400	 lr 7.8125e-05
  train_loss: 0.0032009041169658303
EarlyStopping 317 / 1 counter: 8 out of 500
[20200207T182717] Epock 317 / 400	 lr 7.8125e-05
  train_loss: 0.003190461895428598
EarlyStopping 318 / 1 counter: 9 out of 500
[20200207T182719] Epock 318 / 400	 lr 7.8125e-05
  train_loss: 0.003229873371310532
EarlyStopping 31

[20200207T182933] Epock 373 / 400	 lr 3.90625e-05
  train_loss: 0.0031954755540937185
EarlyStopping 374 / 1 counter: 2 out of 500
[20200207T182936] Epock 374 / 400	 lr 3.90625e-05
  train_loss: 0.003220618120394647
EarlyStopping 375 / 1 counter: 3 out of 500
[20200207T182938] Epock 375 / 400	 lr 3.90625e-05
  train_loss: 0.0032027263659983873
EarlyStopping 376 / 1 counter: 4 out of 500
[20200207T182940] Epock 376 / 400	 lr 3.90625e-05
  train_loss: 0.0031945870723575354
EarlyStopping 377 / 1 counter: 5 out of 500
[20200207T182943] Epock 377 / 400	 lr 3.90625e-05
  train_loss: 0.003183795139193535
Validation loss decreased (0.00318530 --> 0.00318380).  Saving model ...
[20200207T182945] Epock 378 / 400	 lr 3.90625e-05
  train_loss: 0.0031931798439472914
EarlyStopping 379 / 1 counter: 1 out of 500
[20200207T182948] Epock 379 / 400	 lr 3.90625e-05
  train_loss: 0.0031863506883382797
EarlyStopping 380 / 1 counter: 2 out of 500
[20200207T182950] Epock 380 / 400	 lr 3.90625e-05
  train_loss:

In [39]:
model.eval()
total_loss = 0
X_trains = []
y_trains = []
for i, data in enumerate(all_loader):
    X_batch, y_batch = data
    X_batch = X_batch.to(device)
    
    y_trains.append(y_batch.cpu().detach().numpy())
    
    
    with torch.no_grad():
        X_pred = model.encoder(X_batch)
#         loss = criterion(X_pred, X_batch)
#         total_loss = total_loss + loss.item()
        
    X_trains.append(X_pred.cpu().detach().numpy())

# print(total_loss / len(all_loader))

In [55]:
train = pd.DataFrame(np.concatenate(X_trains))
fea_cols = list(train.columns)
fea_cols

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127]

In [56]:
train['label'] = np.concatenate(y_trains)

In [57]:
train

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,label
0,0.000007,0.000003,0.000006,0.000021,0.999939,0.999197,0.999997,0.000009,0.999985,4.865262e-08,...,0.000021,0.000016,0.999992,0.999487,1.0,0.999994,1.0,0.000729,0.999995,118
1,0.000007,0.000003,0.000006,0.000021,0.999939,0.999140,0.999997,0.000008,0.999984,4.504285e-08,...,0.000021,0.000016,0.999992,0.999586,1.0,0.999994,1.0,0.000748,0.999995,155
2,0.000006,0.000002,0.000005,0.000020,0.999946,0.999001,0.999998,0.000007,0.999985,2.735852e-08,...,0.000017,0.000014,0.999994,0.999793,1.0,0.999995,1.0,0.000701,0.999996,117
3,0.000006,0.000002,0.000005,0.000020,0.999946,0.998988,0.999998,0.000007,0.999985,2.754486e-08,...,0.000017,0.000014,0.999994,0.999798,1.0,0.999995,1.0,0.000710,0.999996,113
4,0.000007,0.000003,0.000005,0.000021,0.999942,0.999066,0.999997,0.000007,0.999985,3.535596e-08,...,0.000019,0.000015,0.999993,0.999711,1.0,0.999995,1.0,0.000730,0.999995,114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41395,0.000007,0.000003,0.000006,0.000020,0.999939,0.999250,0.999997,0.000009,0.999985,5.325680e-08,...,0.000022,0.000016,0.999992,0.999359,1.0,0.999994,1.0,0.000715,0.999994,181
41396,0.000007,0.000003,0.000006,0.000020,0.999942,0.999158,0.999997,0.000008,0.999985,4.062645e-08,...,0.000020,0.000015,0.999993,0.999594,1.0,0.999995,1.0,0.000709,0.999995,114
41397,0.000007,0.000003,0.000006,0.000021,0.999938,0.999245,0.999997,0.000009,0.999985,5.478985e-08,...,0.000022,0.000016,0.999992,0.999358,1.0,0.999994,1.0,0.000726,0.999994,122
41398,0.000006,0.000002,0.000005,0.000020,0.999947,0.999003,0.999998,0.000007,0.999985,2.673643e-08,...,0.000017,0.000014,0.999994,0.999795,1.0,0.999995,1.0,0.000694,0.999996,118


In [61]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print('model_ts', model_ts)

initscore_filename = ''
params = {
    'boosting':'gbdt',
#     'boosting':'dart',
#     'boosting':'goss',
    'num_leaves': 7,
#     'max_depth': 3,
    'num_class':num_class,
    'objective': 'multiclass',
    'metric':'multi_logloss',
    'num_threads': 16,
    'learning_rate': 0.01,
    'is_unbalance': True,
#     'scale_pos_weight':200,
    'bagging_fraction':1,
#     'bagging_freq':5,
    'feature_fraction':1,
    'initscore_filename':initscore_filename,
#     'lambda_l1':200,
#     'lambda_l2':20,
    'device_type':'gpu',
#     'tree_learner':'data',

}
print(params)

data_params = {
#     'max_bin':127,
#     'enable_bundle': False,
}
print(data_params)

num_round = 10000
print('num_round:', num_round)

model_ts 20200207T184040
{'boosting': 'gbdt', 'num_leaves': 7, 'num_class': 198, 'objective': 'multiclass', 'metric': 'multi_logloss', 'num_threads': 16, 'learning_rate': 0.01, 'is_unbalance': True, 'bagging_fraction': 1, 'feature_fraction': 1, 'initscore_filename': '', 'device_type': 'gpu'}
{}
num_round: 10000


In [62]:
submit_csv = []
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, random_state=81511991154 % 2**32-1, shuffle=True)

cv = 0
for train_index, valid_index in tqdm_notebook(skf.split(train.index, train['label'].values), total=n_splits, desc = 'CV'):
    
    X_train, X_test = train.loc[train_index, fea_cols], train.loc[valid_index, fea_cols] 
    y_train, y_test = train.loc[train_index,'label'], train.loc[valid_index, 'label']    
    
    print(X_train.shape, X_test.shape)
    
    train_set = lgb.Dataset(X_train, label=y_train, params=data_params)
    val_set = lgb.Dataset(X_test, label=y_test, params=data_params)

    evals_result = {}
    model = lgb.train(params, train_set, num_round, early_stopping_rounds=200, 
                            valid_sets=[train_set, val_set],
                            verbose_eval=50,
                            evals_result=evals_result,
#                             init_model=model,
                           )

    model_tag ='{}_{}_{}_{}'.format(model_ts, cv,
                                 evals_result['valid_1']['multi_logloss'][model.best_iteration-1],
                                 evals_result['training']['multi_logloss'][model.best_iteration-1]
                                )
    print(model_tag)

    joblib.dump(model, 'model/{}.model'.format(model_tag))
    
    
    pred = model.predict(test[fea_cols])

    submission = pd.DataFrame(data=pred)
    submission.index = test.index
    submission.index.name = 'id'
    submission = submission.sort_index()
    submission = submission.groupby('id').mean()

    csv_path = 'submit/{}.csv'.format(model_tag)
    submit_csv.append(csv_path)
    submission.to_csv(csv_path, index=True) 
    
    print(submission.sum(axis=1))
    print(submission)
    cv += 1
#     break

(37260, 128) (4140, 128)
Training until validation scores don't improve for 200 rounds
[50]	training's multi_logloss: 3.89152	valid_1's multi_logloss: 4.02554
[100]	training's multi_logloss: 3.58212	valid_1's multi_logloss: 3.77981
[150]	training's multi_logloss: 3.39063	valid_1's multi_logloss: 3.63879
[200]	training's multi_logloss: 3.25662	valid_1's multi_logloss: 3.549
[250]	training's multi_logloss: 3.15739	valid_1's multi_logloss: 3.48882
[300]	training's multi_logloss: 3.08014	valid_1's multi_logloss: 3.44781
[350]	training's multi_logloss: 3.01747	valid_1's multi_logloss: 3.4184
[400]	training's multi_logloss: 2.96483	valid_1's multi_logloss: 3.39819
[450]	training's multi_logloss: 2.91962	valid_1's multi_logloss: 3.3844
[500]	training's multi_logloss: 2.88061	valid_1's multi_logloss: 3.37492
[550]	training's multi_logloss: 2.8458	valid_1's multi_logloss: 3.36837
[600]	training's multi_logloss: 2.81417	valid_1's multi_logloss: 3.36393
[650]	training's multi_logloss: 2.7845	vali

KeyboardInterrupt: 

In [ ]:
model = DNNModel(input_size=len(fea_cols), dropout_probability=0).to(device)
# model = CNNModel(dropout_probability=0.7).to(device)

In [ ]:
val_loss_min = np.Inf

In [ ]:

# criterion(torch.Tensor([[0,0,1]]), torch.Tensor([[0.1,0.2, 1]]))

In [ ]:
# criterion = nn.CrossEntropyLoss(reduction='mean').to(device)

# # criterion = nn.MultiLabelSoftMarginLoss().to(device)

# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# scheduler = StepLR(optimizer, step_size=50, gamma=1.0)

# trainer = Trainer(model, criterion, optimizer, scheduler, device)



##### LR

In [ ]:
# lr_list = [
# #     (0.1, 20),
#     (0.01, 100),
#     (0.003, 50),
#     (0.001, 50),
#     (0.0003, 50),
#     (0.0001, 50),
#     (0.00003, 50),
#     (0.00001, 50),
#     (0.000005, 50),
# ]


##### Trainer

In [ ]:
total_epoch = 10000
patience = 200
val_loss_min = np.Inf

early_stopping = EarlyStopping(patience=patience, min_epoch=1, verbose=True)
early_stopping.val_loss_min = val_loss_min
early_stopping.best_score = None if val_loss_min==np.Inf else -val_loss_min 

model = DNNModel(input_size=len(fea_cols), dropout_probability=0.3).to(device)

# trainer.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     trainer.optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
optimizer = torch.optim.SGD(model.parameters(), lr=0.03, momentum=0.5)

# trainer.scheduler = StepLR(trainer.optimizer, step_size=50, gamma=1.0)
scheduler = CosineAnnealingWarmRestarts(trainer.optimizer, T_0=500)

criterion = nn.CrossEntropyLoss(reduction='mean').to(device)

trainer = Trainer(model, criterion, optimizer, scheduler, device)


for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
    if os.path.isfile('stop.flag'):
        print(f'{e} stop!')
        break

    train_loss = trainer.train(train_loader, e)

    if e % 1 == 0:
        valid_loss = trainer.eval(val_loader)
#         valid_loss = train_loss

        ts = datetime.now().strftime('%Y%m%dT%H%M%S')
        print(f'[{ts}] Epock {e} / {total_epoch}\t lr {trainer.scheduler.get_lr()[0]}')
        print(f'  train_loss: {train_loss}  valid_loss: {valid_loss}')

        early_stopping(valid_loss, model)

        if early_stopping.early_stop:
            print("\tEarly stopping epoch {}, valid loss {}".format(e, early_stopping.val_loss_min))
            break


model.load_state_dict(torch.load('model/checkpoint.pt'))
val_loss_min = early_stopping.val_loss_min

model_path = 'model/{}_{}'.format(model_ts, val_loss_min)
trainer.save('{}.model'.format(model_path))
print(model_path)


In [ ]:
# total_epoch = 10000

# for lr, patience in lr_list:
#     print(lr, patience)
#     if os.path.isfile('stop.flag'):
#         print('stop!')
#         break
    
#     early_stopping = EarlyStopping(patience=patience, min_epoch=1, verbose=True)
#     early_stopping.val_loss_min = val_loss_min
#     early_stopping.best_score = None if val_loss_min==np.Inf else -val_loss_min 
    
#     trainer.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# #     trainer.optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
# #     trainer.optimizer = torch.optim.SGD(model.parameters(), lr=lr)

#     trainer.scheduler = StepLR(trainer.optimizer, step_size=50, gamma=1.0)
    
#     for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
#         if os.path.isfile('stop.flag'):
#             print(f'{e} stop!')
#             break

#         train_loss = trainer.train(train_loader)
        
#         if e % 1 == 0:
#             valid_loss = trainer.eval(val_loader)
#     #         valid_loss = train_loss

#             ts = datetime.now().strftime('%Y%m%dT%H%M%S')
#             print(f'[{ts}] Epock {e} / {total_epoch}\t lr {trainer.scheduler.get_lr()[0]}')
#             print(f'  train_loss: {train_loss}  valid_loss: {valid_loss}')

#             early_stopping(valid_loss, model)

#             if early_stopping.early_stop:
#                 print("\tEarly stopping epoch {}, valid loss {}".format(e, early_stopping.val_loss_min))
#                 break
            

#     model.load_state_dict(torch.load('model/checkpoint.pt'))
# #     trainer.load('model/checkpoint.pt')
#     val_loss_min = early_stopping.val_loss_min
    
    
#     model_path = 'model/{}_{}'.format(model_ts, val_loss_min)
# #     joblib.dump(model, '{}.model'.format(model_path))
# #     torch.save(model.state_dict(), '{}.pt'.format(model_path))
#     trainer.save('{}.model'.format(model_path))
#     print(model_path)

#     # torch.save(model.state_dict(), f'checkpoint.pt.{train_loss}')
    

In [ ]:
# model = joblib.load('model/20200202T232911_0.7551849484443665.model').cpu()

In [ ]:
# df_test = data_loader_all_v2(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=10, nrows=None)
print(test.shape)
# test[fea_cols] = scaler.transform(test[fea_cols].values)

model.eval()
y_pred = model(torch.Tensor(test[fea_cols].values))    


In [ ]:
y_pred = F.softmax(y_pred)

print(y_pred)

In [ ]:
# model_tag ='{}_{}_{}'.format(model_ts, train_loss, valid_loss)
model_tag ='{}'.format(model_ts)

In [ ]:
submission = pd.DataFrame(data=y_pred.cpu().detach().numpy())
submission.index = test.index
submission.index.name = 'id'
submission = submission.sort_index()
submission = submission.groupby('id').mean()

submission.to_csv('submit/{}.csv'.format(model_tag), index=True) 
model_tag

In [ ]:
submission